In [1]:
import sys
from openai import OpenAI
import httpx

from data.constants import DOCUMENT_FILE_PATH, SYSTEM, PREPROMPT, POSTPROMPT, TEMP, LOCAL_SERVER_URL

document_file_path = str(DOCUMENT_FILE_PATH)
system = str(SYSTEM)
preprompt = str(PREPROMPT)
postprompt = str(POSTPROMPT)
temp = str(TEMP)


In [2]:
try:
    with open(document_file_path, 'r') as file:
        document = file.read()
except FileNotFoundError:
    print(f"Error: The file '{document_file_path}' does not exist.")
    sys.exit(1)
except Exception as e:
    print(f"Error: {e}")
    sys.exit(1)


In [3]:

client = OpenAI(base_url=LOCAL_SERVER_URL, api_key="none", timeout=httpx.Timeout(3600))

completion = client.chat.completions.create(
  model="llama-3.2-3b-it-q8_0",
  messages=[
    {"role": "system", "content": system },
    {"role": "user", "content": preprompt },
    {"role": "user", "content": document },
    {"role": "user", "content": postprompt }
  ],
  temperature=temp,
  stream=True,
)

print(f'question: {postprompt}')
print('\n')
for chunk in completion:
  if chunk.choices[0].delta.content:
    print('model answer:')
    print(chunk.choices[0].delta.content, end="", flush=True)
print('\n')


APITimeoutError: Request timed out.

In [4]:
from pandasai.llm.local_llm import LocalLLM
import requests

In [5]:
# Переопределённый класс для LocalLLM
class CustomLocalLLM(LocalLLM):
    def __init__(self, api_base, model):
        super().__init__(api_base=api_base, model=model)
        self.api_base = api_base
        self.model = model

    def call(self, instruction, value):
        # Объединяем инструкцию и контекст в одно сообщение
        content = f"{instruction}\n\nContext:\n{value}"
        messages = [
            {"role": "user", "content": content}
        ]

        # Формируем данные для запроса
        data = {
            "model": self.model,
            "messages": messages
        }

        # Отправляем POST-запрос к локальной модели
        headers = {"Content-Type": "application/json"}
        response = requests.post(f"{self.api_base}/v1/chat/completions", json=data, headers=headers)

        # Проверяем ответ
        if response.status_code != 200:
            raise ValueError(f"LLM returned an invalid response: {response.text}")

        response_data = response.json()
        if "choices" not in response_data or len(response_data["choices"]) == 0:
            raise ValueError("LLM returned an invalid response format.")

        return response_data["choices"][0]["message"]["content"]


In [8]:
# Инициализация локальной модели
local_model = CustomLocalLLM(api_base=LOCAL_SERVER_URL, model="qwen2.5-coder-14b-instruct-")

# Формирование запроса
instruction = "Generate a molecular structure in SMILES format for a ligand that interacts with the protein sequence."
value = "Protein sequence: MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP..."

# Отправка запроса
try:
    response = local_model.call(instruction=instruction, value=value)
    print("Response from Local LLM:")
    print(response)
except Exception as e:
    print(f"Error while sending request to Local LLM: {e}")

Error while sending request to Local LLM: HTTPConnectionPool(host='10.147.19.239', port=1234): Max retries exceeded with url: /v1/chat/completions (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x167162ad0>, 'Connection to 10.147.19.239 timed out. (connect timeout=None)'))
